In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('./data/train.csv',index_col='index')
train.head()

/home/frodos/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number
index,,,,,,,,,
0,-999,66162.843750,65620.031250,0.0,0.166890,0.276674,2.988714,0.0,44
1,-999,43900.445312,67091.671875,0.0,-0.179967,-0.478795,1.901420,0.0,98
2,-999,37564.019531,26618.890625,55599.0,-0.267820,-0.282087,2.510245,0.0,64
3,-999,39380.027344,48135.425781,19395.0,0.265197,-0.577493,1.215219,0.0,63
4,-999,65040.781250,44018.468750,12930.0,-0.711161,-0.245592,2.973661,0.0,2


In [6]:
for brickId, brick in train.groupby('brick_number'):
    print brickId

In [164]:
# create dictionaries
zPos = np.sort(brick.Z.unique())
zPosDict = dict(enumerate(zPos)) # dictionary of positions (layerNumber => position)
zDiffDict = dict(enumerate(np.diff(zPos))) # dictionaru of distances (layerNumber => distance to the next layer)
layerDict = {v: k for k, v in zPosDict.iteritems() } # dictionary of layer number (position => layerNumber)

# add extra feature - layer number
brick = brick.assign(layerId = brick.Z.map(layerDict))

In [178]:
dx  = 100 # estimation of error along OX axis
dy  = 100 # estimation of error along OY axis
dtx = 0.2
dty = 0.2

## name of colums
cols = ['index', 'X', 'Y', 'TX', 'TY', 'signal', 'r', 'da', 'db', 'parent']

fdf = pd.DataFrame(columns=cols)

for layerId in np.sort(brick.layerId.unique())[0:-1]:
    dz = zDiffDict.get(layerId)

    layer0 = brick.query('layerId == @layerId')
    layer1 = brick.query('layerId == @layerId+1')
    fdf = pd.concat([fdf, pd.DataFrame(getTracks(layer0,layer1,dx,dy,dtx,dty,dz),columns=cols)])

In [179]:
fdf

,index,X,Y,TX,TY,signal,r,da,db,parent
0,5834740.0,69909.320312,74256.406250,0.311124,0.374330,0.0,123.748312,-0.154278,0.205469,610165.0
1,2265329.0,65725.296875,26271.714844,0.215923,-0.455413,0.0,74.005618,-0.039732,-0.195015,2087518.0
2,1376425.0,54370.027344,40487.500000,0.441257,-0.162537,0.0,86.829377,0.175967,-0.158612,2869599.0
3,554911.0,56458.679688,58896.593750,0.487686,0.280394,0.0,48.354133,0.034338,0.068899,3367417.0
4,2823134.0,64875.492188,46833.593750,-0.213690,-0.060286,0.0,13.581616,0.012351,-0.030711,6257570.0
5,4239023.0,65777.984375,55692.339844,0.373028,0.057999,0.0,81.841743,-0.073103,0.252865,9296968.0
0,9652674.0,48256.351562,68183.390625,-0.227865,-0.395647,0.0,35.696873,0.045052,-0.058147,586621.0
1,9710650.0,52451.640625,63391.449219,0.222266,-0.234970,0.0,59.534790,-0.185733,0.252121,4310049.0
2,7373821.0,36648.550781,12939.109375,-0.298400,-0.033594,0.0,77.926880,-0.194262,0.056027,4952692.0
3,5041265.0,63716.562500,29822.179688,0.328051,0.170312,0.0,50.531636,-0.099516,0.471522,6297689.0


In [175]:
pd.DataFrame(tr,columns=cols)

,index,X,Y,TX,TY,signal,r,da,db,parent
0,7637233.0,58299.304688,61676.281250,0.078571,0.181994,0.0,50.020189,0.012909,-0.038430,267649.0
1,3657468.0,58471.523438,41906.664062,0.314007,-0.070633,1.0,27.877151,0.053660,-0.113128,505467.0
2,1372556.0,36038.746094,25046.949219,-0.000874,-0.218601,0.0,65.507177,-0.016118,0.228441,1328847.0
3,7422677.0,58145.371094,41512.699219,0.107041,0.050130,1.0,9.130223,0.039314,-0.025302,1671182.0
4,4068495.0,58230.539062,41500.718750,0.038802,0.210784,1.0,94.249788,-0.028925,0.135352,1671182.0
5,1452673.0,58556.644531,41779.523438,0.162109,0.171387,1.0,77.236650,0.053344,0.073912,1692988.0
6,5222299.0,44905.007812,55833.839844,0.153218,-0.193192,0.0,88.110472,-0.110082,0.323475,2509308.0
7,3273159.0,58442.027344,41701.109375,0.424186,0.242927,1.0,43.060736,0.175567,0.133152,3428599.0
8,7422677.0,58145.371094,41512.699219,0.107041,0.050130,1.0,85.788326,0.066578,-0.151902,4800645.0
9,4068495.0,58230.539062,41500.718750,0.038802,0.210784,1.0,1.735329,-0.001660,0.008752,4800645.0


In [155]:
%%time
def getTracks(layer0,layer1,dx,dy,dtx,dty,dz):
    '''
    layer0, layer1 - Pandas dataframe with track parameters
    dx, dy, dtx, dty - errors of parameters
    dz - distance between layer1 and layer0
    '''

    layer0Arr = layer0.reset_index().as_matrix(columns = ['index', 'X', 'Y', 'TX', 'TY', 'signal'])
    layer1Arr = layer1.sort_values(by='X',ascending=True).reset_index().as_matrix(columns = ['index', 'X', 'Y', 'TX', 'TY', 'signal'])

    res = np.empty([0,10])
    for track in layer0Arr:
        #calculate new pos
        newX  = track[1]+dz*np.tan(track[3])
        newY  = track[2]+dz*np.tan(track[4])
        newTX = track[3]
        newTY = track[4]

        #print 
        lEdge = np.searchsorted(layer1Arr[:,1],newX-dx,side='left')
        rEdge = np.searchsorted(layer1Arr[:,1],newX+dx,side='right')
        subSet = layer1Arr[lEdge:rEdge,:]
        subSet = subSet[np.where((subSet[:,2]>newY-dy) &  (subSet[:,2]<newY+dy) & (subSet[:,3]>newTX-dtx) & (subSet[:,3]<newTX+dtx) & (subSet[:,4]>newTY-dty) & (subSet[:,4]<newTX+dty)),:][0]

        if subSet.shape[0] > 0:
            #print subSet
            r = np.sqrt(np.power(subSet[:,1]-newX,2)+np.power(subSet[:,2]-newY,2))
            da = subSet[:,3]-newTX
            db = subSet[:,4]-newTY
            subSet = np.hstack((subSet,np.expand_dims(r,1),np.expand_dims(da,1),np.expand_dims(db,1),track[0]*np.ones([db.shape[0],1])))
            res = np.vstack((res,subSet))

    return res        

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.87 µs


In [149]:
for layerId in c[0:-1]:
    print layerId

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [163]:
for brickId, brick in train.groupby('brick_number'):
    print brickId

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
